In [1]:
from cold_damp_filter_ctrl import cold_damp_filter_ctrl as filt_ctrl
from biquad_calculator import calcBiquad

In [2]:
#### Cold damping filter initialization
fc = filt_ctrl()
fc.load_bitfile()

/usr/local/anaconda3/envs/python37/lib/python3.7/site-packages/paramiko/ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo
/usr/local/anaconda3/envs/python37/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
/usr/local/anaconda3/envs/python37/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  

ftp_tmp
Bitfile Loaded


In [3]:

fc.set_input(1) #1: particle, 2: MFLI 4968, output.
fc.set_output(1, 'input')
fc.set_output(2, 'input')



In [5]:
#### Cold damping filter setup
cd_output = 'biquad2'
fc.set_output(2, cd_output)
fc.set_output(1, cd_output)

fc.set_input(1) #1: particle, 2: MFLI 4968, output.

fc.set_delay_raw(4)
fc.set_delay_fine(0)

fc.reset_and_start()

fc.set_switch('RF1')

def cd_fc_get_settings(total_gain_db):
    db_left = total_gain_db
    att=0
    if total_gain_db < 10:
        att = 36
        db_left += 36
    amplifier_db = int((db_left+9)/10)*10
    if amplifier_db > 60:
        amplifier_db = 60
    elif amplifier_db < 10:
        amplifier_db = 10
    db_left -= amplifier_db;
    if db_left>0:
        raise NotImplementedError('Too much gain, can\'t do!, Gain=', total_gain_db)
    else:
        add_att_db = -db_left
        return {'Attenuator': att,
                'Amplifier': amplifier_db,
                'AddAtten': add_att_db}

def config_cd_filter(gain, x_notch_freq=146.839e3, y_notch_freq=165.353e3):
    settings = cd_fc_get_settings(gain)
    add_att_linear = 10**(-settings['AddAtten']/20)
    print('Linear Gain in RPi:', add_att_linear)
    print('Amplifier Gain (dB):', settings['Amplifier'])
    print('External Attenuator (dB):', settings['Attenuator'])

    fc.configure_biquad(1, *calcBiquad(biquad_type='notch', Fs=fc.fs, Fc=x_notch_freq, Q=5, gain=-1))
    fc.configure_biquad(2, *calcBiquad(biquad_type='notch', Fs=fc.fs, Fc=y_notch_freq, Q=5, gain=add_att_linear))
    fc.reset_and_start()
    fc.set_amplifier_gain(settings['Amplifier'])
    fc.set_attenuator(settings['Attenuator'])

In [14]:
config_cd_filter(-15, 154.5e3, 190e3)
fc.reset_and_start()

Linear Gain in RPi: 0.35481338923357547
Amplifier Gain (dB): 30
External Attenuator (dB): 36


In [40]:
## for reheating experiments. After executing, the RF switch will switch back and forth 
fc.toggle_switch(187, 0.0215,0.0317)

In [7]:
## Needed to reset after reheating experiments
fc.set_switch('RF1')

In [25]:
(0.0215 + 0.0317)*220

11.703999999999999

In [10]:
750/4

187.5

In [28]:
10/((0.0215 + 0.0317))

187.9699248120301